In [52]:
import pandas as pd
import numpy as np

#from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics

from sklearn.cluster import KMeans, MiniBatchKMeans

import sys
from time import time

from textblob import TextBlob
from textblob.classifiers import NaiveBayesClassifier

import sklearn
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
import nltk

In [69]:
data = pd.read_csv('Tweets.csv')
df2 = data['text']
ltxt = len(df2)
ntrain = ltxt // 3*2
text_train = data['airline_sentiment']

# Countvectorizer & TF-IDF
foovec = CountVectorizer(min_df=20, tokenizer=nltk.word_tokenize)

sents_counts = foovec.fit_transform(df2)
sent_vec=foovec.vocabulary_

# Convert raw frequency counts into TF-IDF (Term Frequency -- Inverse Document Frequency) values
tfidf_transformer = TfidfTransformer()
sents_tfidf = tfidf_transformer.fit_transform(sents_counts)

# Now ready to build a classifier. 
# We will use Multinominal Naive Bayes as our model
from sklearn.naive_bayes import MultinomialNB

# Split data into training and test sets
# from sklearn.cross_validation import train_test_split  # deprecated in 0.18
from sklearn.model_selection import train_test_split
docs_train, docs_test, y_train, y_test = train_test_split(
    sents_tfidf, text_train, test_size = 0.20, random_state = 12)

# Train a Multimoda Naive Bayes classifier
clf = MultinomialNB().fit(docs_train, y_train)
# Predicting the Test set results, find accuracy
y_pred = clf.predict(docs_test)
sklearn.metrics.accuracy_score(y_test, y_pred)

0.7657103825136612

In [ ]:
#SVM Method
import scipy.io

Xtr_dig, Xts_dig, ytr_dig, yts_dig = 